In [1]:
# Initialize Everything First! Run this cell before anything else

# Import required libraries
import re
import json
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
import io

# Chess library - install if needed
try:
    import chess
    import chess.pgn
except ImportError:
    print("Installing python-chess...")
    !pip install python-chess
    import chess
    import chess.pgn

# RealCommentaryGenerator class
class RealCommentaryGenerator:
    """Generates chess commentary based on actual Chess Network and Hikaru patterns"""
    
    def __init__(self):
        pass
    
    def generate_opening_commentary(self, metadata: Dict, moves: List[str]) -> str:
        """Generate opening commentary mixing both styles"""
        opening_moves = " ".join(moves[:6])  # First 6 moves
        
        # Chess Network style intro
        jerry_intro = f"Hi everyone, it's Jerry. I have an excellent game to share with you from {metadata.get('event', 'an online tournament')}. "
        jerry_intro += f"This is a {metadata.get('date', 'recent')} game between {metadata['white']} playing white against {metadata['black']}. "
        
        # Opening analysis
        jerry_analysis = f"We're looking at {opening_moves}. This demonstrates classical opening principles with both sides developing their pieces harmoniously. "
        
        # Hikaru style commentary
        hikaru_comment = f"Now, going into this position, both players are following standard theory, but {metadata['white']} could have also considered other setups. "
        hikaru_comment += f"Chat, this is pretty normal opening play. Like, both sides are just developing and fighting for the center. "
        
        return jerry_intro + jerry_analysis + hikaru_comment
    
    def generate_middlegame_commentary(self, move_data: Dict, critical_move: str = None) -> str:
        """Generate middlegame commentary for critical positions"""
        player = move_data.get('player', 'White')
        
        # Chess Network analysis
        jerry_analysis = f"Now let's examine this position carefully. {player} has just played {critical_move}, and this creates several interesting possibilities. "
        jerry_analysis += f"The key question here is how {player} should continue. If we look at the pawn structure, "
        jerry_analysis += f"we can see that {player} has good piece coordination and central control. "
        
        # Hikaru quick assessment  
        hikaru_comment = f"Chat, this position is actually pretty interesting. Like, {player} has some ideas here, "
        hikaru_comment += f"but the opponent definitely has counterplay. I mean, it's not completely winning, but there are chances. "
        
        # Combined tactical insight
        tactical_note = f"The computer evaluation here shows this is approximately equal, but in a practical game, "
        tactical_note += f"the player with better time management and tactical vision will likely prevail. "
        
        return jerry_analysis + hikaru_comment + tactical_note
    
    def generate_endgame_commentary(self, metadata: Dict, final_moves: List[str]) -> str:
        """Generate endgame commentary"""
        result = metadata.get('result', '1-0')
        winner = 'White' if result == '1-0' else 'Black' if result == '0-1' else 'both players'
        
        # Chess Network endgame principles
        jerry_endgame = f"In this endgame position, we can see the importance of precise technique. "
        jerry_endgame += f"The final moves {' '.join(final_moves[-3:])} demonstrate excellent endgame understanding. "
        jerry_endgame += f"This is exactly how you want to convert a winning position - methodically and accurately. "
        
        # Hikaru result assessment
        hikaru_result = f"And that's the game! {winner} gets the win with some really nice play. "
        hikaru_result += f"Chat, this was actually a pretty instructive game. Like, both players showed good opening preparation, "
        hikaru_result += f"but ultimately {winner} was just more accurate in the critical moments. "
        
        return jerry_endgame + hikaru_result
    
    def analyze_game_for_commentary(self, pgn_string: str) -> Dict:
        """Analyze entire game and generate mixed-style commentary"""
        try:
            # Parse the game
            metadata, moves = self.parse_pgn_basic(pgn_string)
            
            # Extract move strings
            move_strings = []
            for move in moves:
                if move.white_move:
                    move_strings.append(f"{move.move_number}.{move.white_move}")
                if move.black_move:
                    move_strings.append(f"{move.black_move}")
            
            commentary_sections = {
                'metadata': metadata,
                'opening_commentary': self.generate_opening_commentary(metadata, move_strings[:12]),
                'middlegame_commentary': self.generate_middlegame_commentary(
                    {'player': metadata['white']}, 
                    move_strings[12] if len(move_strings) > 12 else "the key move"
                ),
                'endgame_commentary': self.generate_endgame_commentary(metadata, move_strings),
                'full_game_summary': self.generate_game_summary(metadata, len(moves))
            }
            
            return commentary_sections
            
        except Exception as e:
            return {'error': f"Failed to generate commentary: {str(e)}"}
    
    def parse_pgn_basic(self, pgn_string: str) -> Tuple[Dict, List]:
        """Basic PGN parsing for commentary generation"""
        pgn_io = io.StringIO(pgn_string)
        game = chess.pgn.read_game(pgn_io)
        
        if not game:
            raise ValueError("Invalid PGN format")
        
        metadata = {
            'white': game.headers.get('White', 'Unknown'),
            'black': game.headers.get('Black', 'Unknown'), 
            'result': game.headers.get('Result', '*'),
            'date': game.headers.get('Date', 'Unknown'),
            'event': game.headers.get('Event', 'Unknown')
        }
        
        # Simplified move parsing
        moves = []
        board = game.board()
        move_number = 1
        
        for move in game.mainline_moves():
            is_white_move = board.turn == chess.WHITE
            san_move = board.san(move)
            board.push(move)
            
            if is_white_move:
                chess_move = type('ChessMove', (), {
                    'move_number': move_number,
                    'white_move': san_move,
                    'black_move': None
                })()
                moves.append(chess_move)
            else:
                if moves and moves[-1].move_number == move_number:
                    moves[-1].black_move = san_move
                move_number += 1
        
        return metadata, moves
    
    def generate_game_summary(self, metadata: Dict, total_moves: int) -> str:
        """Generate final game summary"""
        result_text = {
            '1-0': f"{metadata['white']} wins",
            '0-1': f"{metadata['black']} wins", 
            '1/2-1/2': "The game ends in a draw"
        }.get(metadata['result'], "Game result unclear")
        
        summary = f"This was an excellent {total_moves}-move game between {metadata['white']} and {metadata['black']} "
        summary += f"from {metadata['event']}. {result_text}. "
        summary += f"Both players demonstrated solid opening preparation and tactical awareness. "
        summary += f"As always, feel free to leave any feedback in the comments section below. "
        summary += f"Hope you enjoyed it and maybe took a thing or two away. That's all for now. Take care!"
        
        return summary

# Initialize the real commentary generator
real_commentator = RealCommentaryGenerator()

# Main function to generate commentary
def generate_chess_commentary(pgn_input: str, style: str = "mixed") -> str:
    """
    Main function to generate chess commentary from PGN
    
    Args:
        pgn_input: PGN string of the chess game
        style: "mixed" (default), "jerry" (Chess Network focus), "hikaru" (Hikaru focus)
    
    Returns:
        Complete commentary as a formatted string
    """
    try:
        result = real_commentator.analyze_game_for_commentary(pgn_input)
        
        if 'error' in result:
            return f"Error: {result['error']}"
        
        # Format the complete commentary
        commentary = f"""
🎬 CHESS COMMENTARY - {result['metadata']['white']} vs {result['metadata']['black']}
{'='*80}

📋 GAME INFO:
   Event: {result['metadata']['event']}
   Date: {result['metadata']['date']}
   Result: {result['metadata']['result']}

🎙️ OPENING ANALYSIS:
{result['opening_commentary']}

⚔️ MIDDLEGAME BREAKDOWN:
{result['middlegame_commentary']}

🏁 ENDGAME EVALUATION:
{result['endgame_commentary']}

📝 FINAL THOUGHTS:
{result['full_game_summary']}

{'='*80}
        """
        
        return commentary.strip()
        
    except Exception as e:
        return f"Failed to generate commentary: {str(e)}"

print("✅ Chess Commentary Generator initialized successfully!")
print("Now run the next cell to see an example, or skip to the last cell to use your own PGN.")

✅ Chess Commentary Generator initialized successfully!
Now run the next cell to see an example, or skip to the last cell to use your own PGN.


In [2]:
# Now use YOUR PGN! Replace the example below with your chess game
your_pgn = """
[Event "Tata Steel Group A"]
[Site "Wijk aan Zee NED"]
[Date "2013.01.15"]
[EventDate "2013.01.12"]
[Round "4"]
[Result "0-1"]
[White "Levon Aronian"]
[Black "Viswanathan Anand"]
[ECO "D47"]
[WhiteElo "2802"]
[BlackElo "2772"]
[PlyCount "46"]

1. d4 d5 2. c4 c6 3. Nf3 Nf6 4. Nc3 e6 5. e3 Nbd7 6. Bd3 dxc4
7. Bxc4 b5 8. Bd3 Bd6 9. O-O O-O 10. Qc2 Bb7 11. a3 Rc8
12. Ng5 c5 13. Nxh7 Ng4 14. f4 cxd4 15. exd4 Bc5 16. Be2 Nde5
17. Bxg4 Bxd4+ 18. Kh1 Nxg4 19. Nxf8 f5 20. Ng6 Qf6 21. h3
Qxg6 22. Qe2 Qh5 23. Qd3 Be3 0-1
"""

print("🎬 Generating Commentary for Your Game...")
print("=" * 60)
commentary = generate_chess_commentary(your_pgn)
print(commentary)

print("\n💡 TIP: Replace the 'your_pgn' variable above with any PGN to get commentary!")

🎬 Generating Commentary for Your Game...
🎬 CHESS COMMENTARY - Levon Aronian vs Viswanathan Anand

📋 GAME INFO:
   Event: Tata Steel Group A
   Date: 2013.01.15
   Result: 0-1

🎙️ OPENING ANALYSIS:
Hi everyone, it's Jerry. I have an excellent game to share with you from Tata Steel Group A. This is a 2013.01.15 game between Levon Aronian playing white against Viswanathan Anand. We're looking at 1.d4 d5 2.c4 c6 3.Nf3 Nf6. This demonstrates classical opening principles with both sides developing their pieces harmoniously. Now, going into this position, both players are following standard theory, but Levon Aronian could have also considered other setups. Chat, this is pretty normal opening play. Like, both sides are just developing and fighting for the center. 

⚔️ MIDDLEGAME BREAKDOWN:
Now let's examine this position carefully. Levon Aronian has just played 7.Bxc4, and this creates several interesting possibilities. The key question here is how Levon Aronian should continue. If we look at 

In [3]:
# Example: Test with a famous game
example_pgn = """
[Event "World Championship Match"]
[Site "Reykjavik ISL"] 
[Date "1972.08.31"]
[Round "6"]
[White "Fischer, Robert J."]
[Black "Spassky, Boris V."]
[Result "1-0"]

1. c4 e6 2. Nf3 d5 3. d4 Nf6 4. Nc3 Be7 5. Bg5 O-O 6. e3 h6 7. Bh4 b6 
8. cxd5 Nxd5 9. Bxe7 Qxe7 10. Nxd5 exd5 11. Rc1 Be6 12. Qa4 c5 13. Qa3 Rc8 
14. Bb5 a6 15. dxc5 bxc5 16. O-O Ra7 17. Be2 Nd7 18. Nd4 Qf8 19. Nxe6 fxe6 
20. e4 d4 21. f4 Qe7 22. e5 Rb8 23. Bc4 Kh8 24. Qh3 Nf8 25. b3 a5 26. f5 exf5 
27. Rxf5 Nh7 28. Rcf1 Qd8 29. Qg3 Re7 30. h4 Rbb7 31. e6 Rbc7 32. Qe5 Qe8 
33. a4 Qd8 34. R1f2 Qe8 35. R2f3 Qd8 36. Bd3 Qe8 37. Qe4 Nf6 38. Rxf6 gxf6 
39. Rxf6 Kg8 40. Bc4 Kh8 41. Qf4 1-0
"""

print("🎯 Example: Fischer vs Spassky, 1972 World Championship")
print("=" * 60)
commentary = generate_chess_commentary(example_pgn)
print(commentary)

🎯 Example: Fischer vs Spassky, 1972 World Championship
🎬 CHESS COMMENTARY - Fischer, Robert J. vs Spassky, Boris V.

📋 GAME INFO:
   Event: World Championship Match
   Date: 1972.08.31
   Result: 1-0

🎙️ OPENING ANALYSIS:
Hi everyone, it's Jerry. I have an excellent game to share with you from World Championship Match. This is a 1972.08.31 game between Fischer, Robert J. playing white against Spassky, Boris V.. We're looking at 1.c4 e6 2.Nf3 d5 3.d4 Nf6. This demonstrates classical opening principles with both sides developing their pieces harmoniously. Now, going into this position, both players are following standard theory, but Fischer, Robert J. could have also considered other setups. Chat, this is pretty normal opening play. Like, both sides are just developing and fighting for the center. 

⚔️ MIDDLEGAME BREAKDOWN:
Now let's examine this position carefully. Fischer, Robert J. has just played 7.Bh4, and this creates several interesting possibilities. The key question here is how 

## Usage Instructions

### How to use this updated chess commentary generator:

1. **Simple Usage**: 
   ```python
   # Just paste your PGN and get commentary!
   your_pgn = "[Event...] 1. e4 e5 2. Nf3..."
   commentary = generate_chess_commentary(your_pgn)
   print(commentary)
   ```

2. **Advanced Usage**:
   ```python
   # Use the RealCommentaryGenerator class directly for more control
   generator = RealCommentaryGenerator()
   result = generator.analyze_game_for_commentary(your_pgn)
   ```

### What's different in this version:

✅ **Real Commentary Patterns** - Based on actual Chess Network and Hikaru transcripts  
✅ **Authentic Language** - Uses actual phrases and expressions from both creators  
✅ **Structured Output** - Generates opening, middlegame, endgame, and summary sections  
✅ **Easy Integration** - Simple function call generates complete commentary  
✅ **Mixed Styles** - Seamlessly blends Jerry's educational approach with Hikaru's entertainment  

### Key Features:

- **Chess Network Style**: "Hi everyone, it's Jerry", detailed analysis, computer evaluations
- **Hikaru Style**: "Chat, this is just winning", quick assessments, casual observations  
- **Game Phase Analysis**: Different commentary styles for opening, middlegame, endgame
- **Realistic Flow**: Natural transitions between analytical and entertaining commentary

In [4]:
# Create an easy-to-use function for generating commentary from any PGN
def generate_chess_commentary(pgn_input: str, style: str = "mixed") -> str:
    """
    Main function to generate chess commentary from PGN
    
    Args:
        pgn_input: PGN string of the chess game
        style: "mixed" (default), "jerry" (Chess Network focus), "hikaru" (Hikaru focus)
    
    Returns:
        Complete commentary as a formatted string
    """
    try:
        result = real_commentator.analyze_game_for_commentary(pgn_input)
        
        if 'error' in result:
            return f"Error: {result['error']}"
        
        # Format the complete commentary
        commentary = f"""
🎬 CHESS COMMENTARY - {result['metadata']['white']} vs {result['metadata']['black']}
{'='*80}

📋 GAME INFO:
   Event: {result['metadata']['event']}
   Date: {result['metadata']['date']}
   Result: {result['metadata']['result']}

🎙️ OPENING ANALYSIS:
{result['opening_commentary']}

⚔️ MIDDLEGAME BREAKDOWN:
{result['middlegame_commentary']}

🏁 ENDGAME EVALUATION:
{result['endgame_commentary']}

📝 FINAL THOUGHTS:
{result['full_game_summary']}

{'='*80}
        """
        
        return commentary.strip()
        
    except Exception as e:
        return f"Failed to generate commentary: {str(e)}"

# Test with a different game
modern_game_pgn = """
[Event "AI Cup 2024"]
[Site "Online"]
[Date "2024.07.15"]
[Round "5"]
[White "Carlsen, Magnus"]  
[Black "Nakamura, Hikaru"]
[Result "1/2-1/2"]

1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Nf6 5. Nc3 a6 6. Be3 e6 7. f3 b5 
8. Qd2 Bb7 9. O-O-O Nbd7 10. h4 b4 11. Nd5 exd5 12. exd5 Ne5 13. Kb1 Rc8 
14. f4 Nc4 15. Bxc4 Rxc4 16. f5 Be7 17. Nf3 O-O 18. Rhe1 Re8 19. Bd4 Bf6 
20. Re2 Rxe2 21. Qxe2 Bxd4 22. Nxd4 Qg5 23. g3 Re4 24. Qf2 Qf6 25. Re1 Rxe1+ 
26. Qxe1 Qxd4 27. Qe8+ Nxe8 28. cxd4 1/2-1/2
"""

print("🎮 TESTING WITH MODERN GAME:")
print("=" * 50)
commentary = generate_chess_commentary(modern_game_pgn)
print(commentary)

illegal san: 'Bf6' in 3qr1k1/1b2bppp/p2p1n2/3P1P2/1prB3P/5N2/PPPQ2P1/1K1RR3 b - - 6 19 while parsing <Game at 0x105562310 ('Carlsen, Magnus' vs. 'Nakamura, Hikaru', '2024.07.15' at 'Online')>


🎮 TESTING WITH MODERN GAME:
🎬 CHESS COMMENTARY - Carlsen, Magnus vs Nakamura, Hikaru

📋 GAME INFO:
   Event: AI Cup 2024
   Date: 2024.07.15
   Result: 1/2-1/2

🎙️ OPENING ANALYSIS:
Hi everyone, it's Jerry. I have an excellent game to share with you from AI Cup 2024. This is a 2024.07.15 game between Carlsen, Magnus playing white against Nakamura, Hikaru. We're looking at 1.e4 c5 2.Nf3 d6 3.d4 cxd4. This demonstrates classical opening principles with both sides developing their pieces harmoniously. Now, going into this position, both players are following standard theory, but Carlsen, Magnus could have also considered other setups. Chat, this is pretty normal opening play. Like, both sides are just developing and fighting for the center. 

⚔️ MIDDLEGAME BREAKDOWN:
Now let's examine this position carefully. Carlsen, Magnus has just played 7.f3, and this creates several interesting possibilities. The key question here is how Carlsen, Magnus should continue. If we look at the pawn structu

In [5]:
# Test the new commentary generator with the Immortal Game
test_pgn = """
[Event "London"]
[Site "London"]
[Date "1851.06.21"]
[Round "?"]
[White "Anderssen, Adolf"]
[Black "Kieseritzky, Lionel"]
[Result "1-0"]

1. e4 e5 2. f4 exf4 3. Bc4 Qh4+ 4. Kf1 b5 5. Bxb5 Nf6 6. Nf3 Qh6 7. d3 Nh5 
8. Nh4 Qg5 9. Nf5 c6 10. g4 Nf6 11. Rg1 cxb5 12. h4 Qg6 13. h5 Qg5 14. Qf3 Ng8 
15. Bxf4 Qf6 16. Nc3 Bc5 17. Nd5 Qxb2 18. Bd6 Bxg1 19. e5 Qxa1+ 20. Ke2 Na6 
21. Nxg7+ Kd8 22. Qf6+ Nxf6 23. Be7# 1-0
"""

print("🎬 Generating Chess Network + Hikaru style commentary...")
print("=" * 60)

# Generate complete commentary
result = real_commentator.analyze_game_for_commentary(test_pgn)

if 'error' not in result:
    metadata = result['metadata']
    
    print(f"🎯 GAME INFO:")
    print(f"   White: {metadata['white']}")
    print(f"   Black: {metadata['black']}")
    print(f"   Event: {metadata['event']}")
    print(f"   Result: {metadata['result']}")
    
    print(f"\n🎙️ OPENING COMMENTARY:")
    print("-" * 40)
    print(result['opening_commentary'])
    
    print(f"\n⚔️ MIDDLEGAME COMMENTARY:")
    print("-" * 40)  
    print(result['middlegame_commentary'])
    
    print(f"\n🏁 ENDGAME COMMENTARY:")
    print("-" * 40)
    print(result['endgame_commentary'])
    
    print(f"\n📝 GAME SUMMARY:")
    print("-" * 40)
    print(result['full_game_summary'])
    
else:
    print(f"❌ Error: {result['error']}")

print("\n" + "=" * 60)

🎬 Generating Chess Network + Hikaru style commentary...
🎯 GAME INFO:
   White: Anderssen, Adolf
   Black: Kieseritzky, Lionel
   Event: London
   Result: 1-0

🎙️ OPENING COMMENTARY:
----------------------------------------
Hi everyone, it's Jerry. I have an excellent game to share with you from London. This is a 1851.06.21 game between Anderssen, Adolf playing white against Kieseritzky, Lionel. We're looking at 1.e4 e5 2.f4 exf4 3.Bc4 Qh4+. This demonstrates classical opening principles with both sides developing their pieces harmoniously. Now, going into this position, both players are following standard theory, but Anderssen, Adolf could have also considered other setups. Chat, this is pretty normal opening play. Like, both sides are just developing and fighting for the center. 

⚔️ MIDDLEGAME COMMENTARY:
----------------------------------------
Now let's examine this position carefully. Anderssen, Adolf has just played 7.d3, and this creates several interesting possibilities. The ke

## Demo: Generate Commentary from PGN

Now let's test the updated commentary generator with a real game!

In [6]:
class RealCommentaryGenerator:
    """Generates chess commentary based on actual Chess Network and Hikaru patterns"""
    
    def __init__(self, api_key: str = None):
        if api_key:
            openai.api_key = api_key
        
        # Real commentary patterns extracted from actual videos
        self.chess_network_patterns = {
            'opening_intro': [
                "Hi everyone, it's Jerry. I have an {adjective} game to share with you from {event}.",
                "We have in this one a {opening_name} opening.", 
                "We're looking at {variation_name}.",
                "{move} marks the {style} followup from here."
            ],
            'move_analysis': [
                "This move demonstrates {concept}. By playing {move}, {player} is {purpose}.",
                "Here we see {mistake_type}. {move} allows {opponent} to {consequence}.",
                "The {principle} are being followed perfectly here.",
                "This is a classic example of why {concept} matters more than {other_concept}.",
                "Notice how {observation}. This is significant because {explanation}."
            ],
            'position_evaluation': [
                "Computer prefers {side} about {evaluation} from here.",
                "Evaluation here is {score} with {top_move} as a top move.",
                "This position is {assessment} for {player}.",
                "The computer says {move} is {quality}."
            ],
            'tactical_moments': [
                "If {hypothetical_move} was played, here's how {player} can respond: {sequence}",
                "Let me show you what this looks like. {explanation}",
                "Watch what happens next: {tactical_sequence}",
                "This is where we could see {tactic} with {follow_up}."
            ]
        }
        
        self.hikaru_patterns = {
            'opening_chat': [
                "Welcome back everyone. For today's video, we are going to be taking a look at {event}.",
                "Chat, {observation}. Like, {assessment}.",
                "Now rather shockingly, {surprising_fact}.",
                "Going into this {round} round matchup, {situation}."
            ],
            'move_reactions': [
                "Chat, this is just winning for {player}. Like, {opponent} probably didn't see this coming at all.",
                "Oh no, {player} just blundered! This is so bad. {move} just loses the game immediately.",
                "Wait, this is actually genius. {player} is {action} but {reason}.",
                "This position is so good for {player}. Like, it's not even close.",
                "Actually brilliant! {player} saw this like {moves} moves ago."
            ],
            'quick_assessments': [
                "This should be a draw with correct play.",
                "Yeah, {player}'s going to {result} now.",
                "I mean, this is just {assessment}.",
                "That's actually a good move.",
                "Little bit confused by that.",
                "Very strange. I'm {reaction}."
            ],
            'casual_observations': [
                "I mean, all the openings are fine. Depends what kind of game you want.",
                "It's definitely not {place}. I'll tell you guys that.",
                "Thanks so much to {user} for the Prime. Thank you so much.",
                "Let's leave it at that.",
                "Um, I'm not sure I want to answer that question."
            ]
        }
    
    def generate_opening_commentary(self, metadata: Dict, moves: List[str]) -> str:
        """Generate opening commentary mixing both styles"""
        opening_moves = " ".join(moves[:6])  # First 6 moves
        
        # Chess Network style intro
        jerry_intro = f"Hi everyone, it's Jerry. I have an excellent game to share with you from {metadata.get('event', 'an online tournament')}. "
        jerry_intro += f"This is a {metadata.get('date', 'recent')} game between {metadata['white']} playing white against {metadata['black']}. "
        
        # Opening analysis
        jerry_analysis = f"We're looking at {opening_moves}. This demonstrates classical opening principles with both sides developing their pieces harmoniously. "
        
        # Hikaru style commentary
        hikaru_comment = f"Now, going into this position, both players are following standard theory, but {metadata['white']} could have also considered other setups. "
        hikaru_comment += f"Chat, this is pretty normal opening play. Like, both sides are just developing and fighting for the center. "
        
        return jerry_intro + jerry_analysis + hikaru_comment
    
    def generate_middlegame_commentary(self, move_data: Dict, critical_move: str = None) -> str:
        """Generate middlegame commentary for critical positions"""
        player = move_data.get('player', 'White')
        
        # Chess Network analysis
        jerry_analysis = f"Now let's examine this position carefully. {player} has just played {critical_move}, and this creates several interesting possibilities. "
        jerry_analysis += f"The key question here is how {player} should continue. If we look at the pawn structure, "
        jerry_analysis += f"we can see that {player} has good piece coordination and central control. "
        
        # Hikaru quick assessment  
        hikaru_comment = f"Chat, this position is actually pretty interesting. Like, {player} has some ideas here, "
        hikaru_comment += f"but the opponent definitely has counterplay. I mean, it's not completely winning, but there are chances. "
        
        # Combined tactical insight
        tactical_note = f"The computer evaluation here shows this is approximately equal, but in a practical game, "
        tactical_note += f"the player with better time management and tactical vision will likely prevail. "
        
        return jerry_analysis + hikaru_comment + tactical_note
    
    def generate_endgame_commentary(self, metadata: Dict, final_moves: List[str]) -> str:
        """Generate endgame commentary"""
        result = metadata.get('result', '1-0')
        winner = 'White' if result == '1-0' else 'Black' if result == '0-1' else 'both players'
        
        # Chess Network endgame principles
        jerry_endgame = f"In this endgame position, we can see the importance of precise technique. "
        jerry_endgame += f"The final moves {' '.join(final_moves[-3:])} demonstrate excellent endgame understanding. "
        jerry_endgame += f"This is exactly how you want to convert a winning position - methodically and accurately. "
        
        # Hikaru result assessment
        hikaru_result = f"And that's the game! {winner} gets the win with some really nice play. "
        hikaru_result += f"Chat, this was actually a pretty instructive game. Like, both players showed good opening preparation, "
        hikaru_result += f"but ultimately {winner} was just more accurate in the critical moments. "
        
        return jerry_endgame + hikaru_result
    
    def analyze_game_for_commentary(self, pgn_string: str) -> Dict:
        """Analyze entire game and generate mixed-style commentary"""
        try:
            # Parse the game
            metadata, moves = self.parse_pgn_basic(pgn_string)
            
            # Extract move strings
            move_strings = []
            for move in moves:
                if move.white_move:
                    move_strings.append(f"{move.move_number}.{move.white_move}")
                if move.black_move:
                    move_strings.append(f"{move.black_move}")
            
            commentary_sections = {
                'metadata': metadata,
                'opening_commentary': self.generate_opening_commentary(metadata, move_strings[:12]),
                'middlegame_commentary': self.generate_middlegame_commentary(
                    {'player': metadata['white']}, 
                    move_strings[12] if len(move_strings) > 12 else "the key move"
                ),
                'endgame_commentary': self.generate_endgame_commentary(metadata, move_strings),
                'full_game_summary': self.generate_game_summary(metadata, len(moves))
            }
            
            return commentary_sections
            
        except Exception as e:
            return {'error': f"Failed to generate commentary: {str(e)}"}
    
    def parse_pgn_basic(self, pgn_string: str) -> Tuple[Dict, List]:
        """Basic PGN parsing for commentary generation"""
        pgn_io = io.StringIO(pgn_string)
        game = chess.pgn.read_game(pgn_io)
        
        if not game:
            raise ValueError("Invalid PGN format")
        
        metadata = {
            'white': game.headers.get('White', 'Unknown'),
            'black': game.headers.get('Black', 'Unknown'), 
            'result': game.headers.get('Result', '*'),
            'date': game.headers.get('Date', 'Unknown'),
            'event': game.headers.get('Event', 'Unknown')
        }
        
        # Simplified move parsing
        moves = []
        board = game.board()
        move_number = 1
        
        for move in game.mainline_moves():
            is_white_move = board.turn == chess.WHITE
            san_move = board.san(move)
            board.push(move)
            
            if is_white_move:
                chess_move = type('ChessMove', (), {
                    'move_number': move_number,
                    'white_move': san_move,
                    'black_move': None
                })()
                moves.append(chess_move)
            else:
                if moves and moves[-1].move_number == move_number:
                    moves[-1].black_move = san_move
                move_number += 1
        
        return metadata, moves
    
    def generate_game_summary(self, metadata: Dict, total_moves: int) -> str:
        """Generate final game summary"""
        result_text = {
            '1-0': f"{metadata['white']} wins",
            '0-1': f"{metadata['black']} wins", 
            '1/2-1/2': "The game ends in a draw"
        }.get(metadata['result'], "Game result unclear")
        
        summary = f"This was an excellent {total_moves}-move game between {metadata['white']} and {metadata['black']} "
        summary += f"from {metadata['event']}. {result_text}. "
        summary += f"Both players demonstrated solid opening preparation and tactical awareness. "
        summary += f"As always, feel free to leave any feedback in the comments section below. "
        summary += f"Hope you enjoyed it and maybe took a thing or two away. That's all for now. Take care!"
        
        return summary

# Initialize the real commentary generator
real_commentator = RealCommentaryGenerator()
print("✅ Real Commentary Generator initialized with authentic patterns!")

✅ Real Commentary Generator initialized with authentic patterns!


In [7]:
# Chess Commentary Generator - Updated Version

# Clear the existing workflow and create a new one based on real commentary patterns
print("🔄 Updating Chess Commentary Generator with real style patterns...")
print("=" * 60)

🔄 Updating Chess Commentary Generator with real style patterns...


## IMPORTANT: Run cells in order!

**To use this notebook correctly:**
1. First run the cell below to initialize all classes
2. Then run the example cell to see it in action
3. Finally, use your own PGN in the last cell

## Using the Commentary Generator

### How to use this notebook:

1. **Load your PGN game**: Replace the `sample_pgn` variable with your chess game in PGN format
2. **Choose commentary style**: 
   - `"educational"` - More Chess Network style, focuses on teaching
   - `"entertaining"` - More Hikaru style, focuses on entertainment  
   - `"balanced"` - Combination of both styles
3. **Generate commentary**: Use `workflow.generate_full_commentary(pgn, style)`
4. **Get AI responses**: Set up OpenAI API key and use `workflow.generate_live_commentary(prompt, use_ai=True)`

### Features included:

✅ **PGN Parsing** - Converts chess notation to structured data  
✅ **Move Analysis** - Identifies critical moves, captures, checks  
✅ **Game Phase Detection** - Separates opening, middlegame, endgame  
✅ **Style Templates** - Chess Network + Hikaru personality prompts  
✅ **Commentary Generation** - Creates contextual prompts for AI  
✅ **Flexible Workflow** - Supports different commentary intensities

In [8]:
# Generate commentary for the sample game
print("Generating commentary for The Immortal Game...")
print("=" * 50)

# Generate commentary with different styles
styles = ["educational", "entertaining", "balanced"]

for style in styles:
    print(f"\n🎯 STYLE: {style.upper()}")
    print("-" * 30)
    
    result = workflow.generate_full_commentary(sample_pgn, commentary_style=style)
    
    if 'error' in result:
        print(f"Error: {result['error']}")
        continue
    
    # Display game info
    metadata = result['metadata']
    print(f"📋 Game: {metadata['white']} vs {metadata['black']}")
    print(f"📅 Date: {metadata['date']}")
    print(f"🏆 Result: {metadata['result']}")
    print(f"📍 Event: {metadata['event']}")
    
    # Display phases analysis
    phases = result['game_phases']
    print(f"\n🎮 Game Phases:")
    print(f"   Opening: moves {phases['opening']}")
    print(f"   Middlegame: moves {phases['middlegame']}")
    print(f"   Endgame: moves {phases['endgame']}")
    
    # Display commentary for first few moves as example
    print(f"\n💬 Commentary Examples ({style} style):")
    for i, commentary in enumerate(result['commentary'][:3]):  # Show first 3
        print(f"\n   Move {commentary['move_number']}: {commentary['white_move']} {commentary['black_move'] or ''}")
        print(f"   Phase: {commentary['phase']} | Critical: {commentary['is_critical']}")
        print(f"   Type: {commentary['move_type']} | Intensity: {commentary['intensity']}")
        print(f"   \n   📝 Commentary Prompt Preview:")
        # Show first 200 characters of the prompt
        prompt_preview = commentary['commentary_prompt'][:200] + "..." if len(commentary['commentary_prompt']) > 200 else commentary['commentary_prompt']
        print(f"   {prompt_preview}")
    
    print(f"\n   📊 Total commentary entries generated: {len(result['commentary'])}")
    print("=" * 50)

Generating commentary for The Immortal Game...

🎯 STYLE: EDUCATIONAL
------------------------------


NameError: name 'workflow' is not defined

In [ ]:
# Example PGN game - Famous "Immortal Game" by Adolf Anderssen
sample_pgn = """
[Event "London"]
[Site "London"]
[Date "1851.06.21"]
[Round "?"]
[White "Anderssen, Adolf"]
[Black "Kieseritzky, Lionel"]
[Result "1-0"]

1. e4 e5 2. f4 exf4 3. Bc4 Qh4+ 4. Kf1 b5 5. Bxb5 Nf6 6. Nf3 Qh6 7. d3 Nh5 
8. Nh4 Qg5 9. Nf5 c6 10. g4 Nf6 11. Rg1 cxb5 12. h4 Qg6 13. h5 Qg5 14. Qf3 Ng8 
15. Bxf4 Qf6 16. Nc3 Bc5 17. Nd5 Qxb2 18. Bd6 Bxg1 19. e5 Qxa1+ 20. Ke2 Na6 
21. Nxg7+ Kd8 22. Qf6+ Nxf6 23. Be7# 1-0
"""

print("Sample PGN loaded - The Immortal Game!")

## Example Usage and Demonstration

Below is a complete example showing how to use the chess commentary workflow with a sample game.

In [ ]:
class ChessCommentaryWorkflow:
    """Main workflow class that orchestrates the commentary generation"""
    
    def __init__(self, api_key: str = None):
        self.parser = ChessGameParser()
        self.commentator = ChessCommentaryGenerator(api_key)
        
    def generate_full_commentary(self, pgn_string: str, commentary_style: str = "balanced") -> Dict:
        """
        Generate complete commentary for a chess game
        
        Args:
            pgn_string: PGN format game string
            commentary_style: "educational", "entertaining", "balanced"
            
        Returns:
            Dictionary with game data and commentary
        """
        try:
            # Parse the game
            metadata, moves = self.parser.parse_pgn(pgn_string)
            moves = self.parser.analyze_move_criticality(moves)
            
            # Analyze game phases
            phases = self.commentator.analyze_game_phases(moves)
            
            # Generate commentary for key moves
            commentary_data = {
                'metadata': metadata,
                'moves': [],
                'commentary': [],
                'game_phases': phases
            }
            
            # Select moves to comment on based on importance
            moves_to_comment = self._select_key_moves(moves, commentary_style)
            
            for move_idx in moves_to_comment:
                move = moves[move_idx]
                
                # Determine game phase
                current_phase = "middlegame"  # default
                for phase, indices in phases.items():
                    if move_idx in indices:
                        current_phase = phase
                        break
                
                # Generate commentary prompt
                commentary_prompt = self.commentator.generate_move_commentary(
                    move, metadata, current_phase
                )
                
                commentary_entry = {
                    'move_number': move.move_number,
                    'white_move': move.white_move,
                    'black_move': move.black_move,
                    'phase': current_phase,
                    'is_critical': move.is_critical,
                    'move_type': move.move_type,
                    'commentary_prompt': commentary_prompt,
                    'intensity': self.commentator.get_commentary_intensity(move)
                }
                
                commentary_data['commentary'].append(commentary_entry)
            
            return commentary_data
            
        except Exception as e:
            return {'error': f"Failed to generate commentary: {str(e)}"}
    
    def _select_key_moves(self, moves: List[ChessMove], style: str) -> List[int]:
        """Select which moves to generate commentary for"""
        key_moves = []
        
        for i, move in enumerate(moves):
            # Always comment on critical moves
            if move.is_critical:
                key_moves.append(i)
            # Comment on opening moves (first 8-10 moves)
            elif i < 8:
                key_moves.append(i)
            # Comment on endgame (last few moves)
            elif i >= len(moves) - 5:
                key_moves.append(i)
            # Selective middlegame commentary
            elif style == "educational" and i % 3 == 0:
                key_moves.append(i)
            elif style == "entertaining" and (move.move_type != "normal" or i % 4 == 0):
                key_moves.append(i)
            elif style == "balanced" and i % 5 == 0:
                key_moves.append(i)
        
        return key_moves
    
    def generate_live_commentary(self, commentary_prompt: str, use_ai: bool = False) -> str:
        """
        Generate actual commentary text using AI or return the prompt
        
        Args:
            commentary_prompt: The prepared prompt for commentary
            use_ai: Whether to use AI API to generate actual commentary
            
        Returns:
            Commentary text
        """
        if not use_ai:
            return f"[Commentary Prompt Ready]\n{commentary_prompt}"
        
        try:
            # This would use OpenAI API if available
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a chess commentator combining the educational style of Chess Network with the entertaining personality of GM Hikaru Nakamura."},
                    {"role": "user", "content": commentary_prompt}
                ],
                max_tokens=200,
                temperature=0.7
            )
            return response.choices[0].message.content
        except:
            return f"[AI Commentary Unavailable - Prompt Ready]\n{commentary_prompt}"

# Initialize the workflow
workflow = ChessCommentaryWorkflow()
print("Complete chess commentary workflow ready!")

In [ ]:
class ChessCommentaryGenerator:
    """Generates chess commentary combining Chess Network and Hikaru styles"""
    
    def __init__(self, api_key: str = None):
        if api_key:
            openai.api_key = api_key
        
        # Commentary style templates
        self.chess_network_examples = [
            """
                PGN 1. d4 Nf6 2. c4 e6 3. Nc3 Bb4 4. Qc2 d5 5. a3 Bxc3+ 6. Qxc3 O-O 7. Nf3 Nbd7 8. Bg5 b6 9. cxd5 exd5 10. e3 c5 11. Bb5 h6 12. Bh4 a5 13. Bc6 Ra7 14. dxc5 bxc5 15. Bxd7 Rxd7 16. Qxc5 Ba6 17. Qd4 g5 18. Bg3 Ne4 19. Rc1 Qa8 20. Qd1 Rc8 21. Nd4 Nc5 22. Rc2 a4 23. Kd2 Rb7 24. Qc1 Bc4 25. f3 Qa5+ 26. Kd1 Nb3 27. Be1 Qa6
                Hi everyone, it's Jerry. I have an excellent game to share with you from round four of the 2025 Super United Rapid and Blitz Croasia. Say that five times fast. Rapid game here. 25 minutes 10-second increment on the white end. Notbeck Abdul Satarov playing against Gashd. We have in this one a D pawn based opening with the move bishop B4. We're looking at Nimzo Indian defense. Queen C2 marks the classical followup from here D5 controlling E4 preventing white from forming a pawn trio. You could castle here in respond to E4 with D5. But Gh, let's just say doesn't want to see that trio. So D5 it is A3 bishop for knight exchange and now castles. So bishop pair for white. Slight lead in development for black. White has yet to move a single piece or pawn on the king's side until now. Knight's connected anticipating this pin, this pressure on F6. Got to complete development. Bishop needs a new door or a door. B6 it is. And note the timing here. Tension released as soon as a hold is created on C6. C takes D. E takes D. Queen has access to C6. E3 is played here. I wondered about this queen C6 move, how it plays out here. We have E3 and then C5. Queen after the white queen after C5 no longer has access to C6. What is black doing if queen C6 is played? Rook is targeted. Here's how black can respond. develop the bishop on this diagonal. Queen defends the rook and black needs to be prepared to maybe lose this pawn. So if white piles up like this, you could put the rook on this halfopen file. And if white goes in for this pawn, it's not a good idea. Let me show you what this looks like. Let me show you how uh black's lead in development can compensate for being down a pawn. If white takes here on C7, we trade queens and in the end, black is now going to have full control over this only opened file. In white's development remains backward here on the king's side. Evaluation here is minus.6 six with king D1 as a top move preventing rook entries. It's looking at king D1, knight E4 defense over F2 and then this idea here to bother the king. It's trouble for white. Okay, white does not plant the queen on the C6 square. Plays E3. Forget about queen C6. C5 is now on board. Bishop B5. And from here, bishop questioned. Bishop H4. And now we see A5. How the computer wants to play it. It says go ahead break this pin and then hop in the center. Play here is with A5. Uh, a couple things going on with A5. This does pave the way for bishop A6 and it does give the rook a little more air. Has a new option here. It could be, you know, there could be some cases where the rook maybe gets trapped with the bishops here. So, bishop C6, rook B8, bishop G3. Seems like the rook is out of squares there if not for a timely knight E4 move to track that dark square bishop. Okay, at the end of the day, I believe A5 is primarily there to support bishop A6. This here just a a side benefit followup from here, bishop C6. After bishop C6, we have rook A7. And then white goes ahead and captures on C5. We see a series of exchanges from that point on. The computer says capture on C5. Straight away I want to draw some attention to this immediate capture. This is how a computer wants to run it. Take on C5. Take on D7. And note in this position, black has only two ways to recapture. Knight here pinned. So take with the queen or the bishop. You take with the queen, your king side gets destroyed. So you'd have to take with the bishop. And now it wants to take on C5. White is up a pawn. And uh note in this position there isn't a way for this bishop to easily at least get on this diagonal. Try and prevent white from castling king side. It runs a little deeper. this line here. Rook C8. Queen here preventing a bishop move. If you could get a queen exchange in, that would be nice. It runs a little deeper. Rook C6. Bishop takes knight. Queen C8 at the end of this line here. White is up two pawns. But uh how do you complete development? Uh it could be the case that black uh establishes a major piece on the second rank that should compensate for one of the pawns. Another detail here is that there is a bishop c6 move right around the corner. So for instance, if you castle here, bishop c6, there's a problem here on f3. So it's a pretty deep line for sure. how white runs it does not go straight in for the capture on C5. Inserts first bishop C6. Notice how things are different after rook A7 and the capture on C5. Capture on D7. Ah, a new option is now available to black. A rook can recapture on D7. And that's how black plays it. This is significant. Watch what happens next. Rook takes bishop. Queen takes C5. And now this bishop can get on this beautiful diagonal. Black is down a pawn. But is for choice. Computer prefers black about minus.5 from here. Queen D4. Increased pressure on the knight. That's addressed. Pin broken. Knight now free to move. Hops in the center. Note the squares these miners are taking away from the king. Tough times ahead for white. Now from here, rook C1 is played. If white tries to castle queen side, can't go king side. You go queen side. Are you safer on the queen side than in the center? Probably not. There are these open lines. So, uh, black here, in fact, can start with, uh, a pretty slick move. Bishop E2. Where does this rook go? If you want to hang out in the material, uh, you barely have a square. If you don't want bishop captures knight or knight captures rook, you'd have to go here. That's not any fun. Rook B7. A queen move next. Rook C8. You're not going to survive this. These rooks are very clumsy here. Okay, no queen side castles uh in this position. Again, rook C1 at the top. This is how the computer wants to play it. It wants to do something about this knight potentially wants to start off with knight to E5. And depending on where this rook goes to address this threat, uh white may or may not uh challenge this knight next. So, let me draw attention to the two rook moves in response to this top move, knight E5. If the rook plays to B7, F3 is a good followup to try and get rid of this knight. If however, rook C7 is played, it says knight G6 is the best way forward, targeting both rooks. We'll have a look at that line. But what I want to show you here is why in this position F3 would be a losing move. There would be queen C8 here. You could ignore the pressure on the knight. And I C1 C2. Black is just moments away from tying in a major piece with the minors. Rook C2 to E2 could be an idea. How would this look if the knight is picked off? Rook C1 check. If you move the king, we'll be having mate shortly on E2. And if you take the rook, you're going to lose the rook. Blocking this check will lose the king. Queen takes E3. Queen captures queen next for mate. So how the computer wants to run it if knight E5 and rook C7 was played wants to go for this double attack on the rooks. Pawn takes knight bishop takes rook very forced sequence and in the end queen f7 with a queen exchange next taking the knight will be losing queen captures f2. These three will uh give the king a problem. Wants to go in for this uh queen exchange and play the two minor pieces versus a rook and two pawns imbalance. It's a very sharp line right there. Okay. How it is played here. We don't see the top move. Knight E5. Rook on an uh rook on an open file. followup. Queen A8 paving the way for rook C8 from here. Queen D1 is played. Computer says challenge this knight now with knight D2. Okay, the queen backs off to D1 in this position. What move would you play here? If you'd like to go ahead, pause the video. Uh give this one a think. Okay, Gesh's move. He goes ahead with rook to C8. Now, this is not top. Uh, queen D1 is in fact a blunder. Doesn't take advantage of it. The top move here. Did you spot this one? It says get rid of your isolated pawn. D4 is winning. What's the point? Well, a couple things to note. After the knight recaptures, the knight is now in a pin. Also, with no pawn on D5, the queen has more possibilities on this diagonal. Follow up from here, it says, put this rook on E8. This is really big trouble for white. I mean, what's your move here? Queen can't go too far. She has a response. You want to get out of the pin in such a position. But if you go to a square and you're not eyeing that knight, there's an exchange sacrifice, a follow-up knight move with a discovered check. If you try to question the knight here, there's knight takes bishop. Rook captures E3. That's going to be a disaster. Okay, this was an opportunity in short to pitch the pawn and extend the scope of a couple majors. Rook C8 is played here. Knight gets to blockade. Knight C5. The times for each player at this stage were 21 moves in. They have about 7 minutes each. From here, rook C2 A4. They're now down to about 4 minutes each. Still have that 10-second increment. And from here, white plays king to D2, trying to sort of make a run for it towards the queen side. That's not going to work out. We'll see how black punishes that at the top. This is how the computer wants to play it. Very difficult, you know, facing this possibility of a knight d3 check to play in this way. Computer says try to pry open the H file. play H4 and should there be this check here on D3, king D2, you exchange rooks, rook B7, it says you can hold this apparently defend B2 with rook to B1, which makes you sort of question this H4 idea. If the rook isn't on the H file, why you trying to open up the H file here? It says this is holdable. Now, it's still not going to be a fun time ahead here for white. Black has still this excellent grip on the light squares. The extra pawn white has call it the B pawn. Not a very valuable pawn. All right. Says it's holdable. Good luck defending that though. How does white play it? We don't see H4. King tries to make a run for it. Black's response. Well, let me throw this one to you as a pop quiz. What would your move be here? Feel free to pause the video. Okay, the top move was played here. I love this move. Rook B7. With rook B7, it calls it about minus3. Watch how quickly this one can slip away from black if he's not accurate at this moment. If let's say knight e4 is played, knight e4 check, white slips away. King c1, king b1. It says you can't get at the king anymore. Also, if the knight hops into d3, white could exchange rooks, collect the a pawn. Queen very active now, placing a restriction on black's pieces here. says white can hold this position as well. How are things different with this top move rook B7? Well, it sort of makes life very difficult for white. I mean, look at this piece on C5. Where's it going to go? E4. D3. I didn't even touch on B3 yet. It is very difficult to handle this piece. Also, bishop D3. could be a move in some cases with rook B7. Black is sort of saying, "Uh-uh, I know you're trying to slip away, but now you're going to get hurt. You play king C1. I got a move up my sleeve." How white plays it here, it's queen C1. If king C1 now knight D3 with check, king B1, rook takes rook. Rook takes B2. Queen's lost. Queen versus rook and knight position ahead. Winning for black. So don't try to sneak away with king C1. I have knight D3. Queen C1 played here. If F3 is played, cutting out a knight check on E4. This is where we could see bishop D3 kicking the rook from its defense of B1. This helps us understand why queen C1 is played in the game. Some control over B2. White does not have a sneaky way out here going for the two minor pieces for the rook because in the end, wow, look at these major pieces. You are not surviving this. Okay, the try in this game after rook B7, queen C1, there's a pin. Now, if you're going to move the knight, better come with check. Otherwise, rook captures rook. That's going to collapse. How to break the pin? Bishop C4. Now the knight is ready to go to E4. That's prevented. A nice followup to bishop C4. Queen A5 check now available. What a crushing position this is. King takes a step back. And now the knight hops into B3 with the fork. White's play from here is bishop to E1. The queen is under fire here. If you take the knight, that's going to be painful. You walk right into a pin. What's tried here? Bishop E1. And after the queen takes a step back here to A6. White resigns. What are you supposed to do if the game continued? Knight takes knight. Same thing. We got that pin. Queen is under fire. She barely has a move because of this beast on B3. If you go to B1, there's knight takes knight. And there we go. We get that bishop B3 move in there. Crushing pin. Queen D3 in the air. You don't survive this. This was an excellent game. I know it's only rapid format, but winning with black here. Nimzo Indian 29 mover. Really nice play. Anyhow, your thoughts with this one. Feel free as usual to leave any feedback to this video in the comment section below. I hope you enjoyed it and maybe took a thing or two away. That's all for now. Take care. Bye. 
            """,
            """
                PGN 1. Nf3 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. d4 Bg4 6. Be2 O-O 7. O-O Nc6 8. d5 Bxf3 9. Bxf3 Ne5 10. Be2 c6 11. Be3 cxd5 12. exd5 Rc8 13. b3 Ned7 14. Rc1 Qa5 15. Re1 a6 16. Bd2 Rfe8 17. Bf1 Qb6 18. Rc2 Qd4 19. h3 Rc7 20. Qc1 Qa7 21. Be3 Qb8 22. Bd4 Nh5 23. Qd1 Bxd4 24. Qxd4 Nhf6 25. Rce2 Qd8 26. Re3 Rc8 27. Kh2 Kf8 28. g4 Qb6 29. Qxb6 Nxb6 30. Kg3 Rc7 31. h4 h6 32. Be2 Kg7 33. g5 hxg5 34. hxg5 Nh7 35. f4 Nc8 36. Bg4 b5 37. cxb5 axb5 38. Nxb5 Rc5 39. a4 Nb6 40. Nxd6 Rf8 41. Rxe7 Nxd5 42. R7e5 Rc3+ 43. Bf3 Nb4 44. R1e3 Rc1 45. Re7 Rc5 46. Rb7 Nd5 47. Re5
                Hi everyone, it's Jerry. I have an instructive game to share with you from the ongoing Chinese Rapid and Blitz Championship. This is not the proper championship event. Uh this one here, a 24 player single elimination event. Rapid format for this one. 15 minutes 10-second increment played on July 13th on the white ending playing against Jiao Kuwanha. Okay, this one kicks off with knight f3, knight f6 and the opening we are looking at a kings Indian defense and with bishop g4 this is one of the uh rare lines of the kings Indian defense. We will soon see a bishop for knight exchange and I would say a model game here for the white side playing with this particular imbalance. So we see a same sides castles game. Knight c6 d5 bishop for knight exchange. There we go. And after this break here with C6 and the capture, let me throw this one to you as a pop quiz. What is your move here? Feel free to pause the video. Okay. How to recapture on D5? If you take with the knight, that hangs E4. So, which of these two should recapture? In the game, it's E captures D. And this is the better way to recapture. Uh if you take with the C pawn, this rook once positioned on the opened now opened C file will have a greater influence. It will see C3 directly and also support knight C4. So in this position after rook C8, notice you can't keep the knight out of C4, at least with a pawn that would hang uh the knight. White anticipates the rook C8 move in this game. Takes with the E pawn and after rook C8 connect four available. The knight shielded C pawn still around. Now the big question for black how to get in this one key pawn break B5. I mean there's another break with E6, but this is really the main one to try to get in B5. and white never really allows black to get that break in and maintains excellent uh control throughout this whole game. So let's see how he manages to do this. Uh first knight E D7 rook C1 very healthy queen A5 rook E1 A6 and now bishop D2. So, taking some care over the C3 point now that the B5 move is prepped from here, black plays rook F to E8. Now, let's test out the B5 move in this position. It certainly looks scary. uh the bishop opposite the queen. But let's get a feel for what tricks are up white's sleeve in the event of this break. This move that tries to free black's position and extend the scope of the rook. If B5 was played, here's how white can respond. Likes to first capture on B5 with the pawn, opening up this C4 square. We'll see why soon enough. And after this recapture, now we can uh go ahead and capture on B5 with the knight. And here's the point. If the queen captures A2, there's this nice move, bishop C4, which is sort of the uh will it will act as the superlue for white's position. holds everything together and now carries this last move would carry two threats. A very direct one swiping a pawn and the other is to trap the queen with rook A1 and bishop C3. So there wouldn't be a good answer uh to this. So not a good idea to try for this break. It's simply not there in the game. Rook F to E8. Bishop F1. Just getting out of the rook's way. Some more control over a possible E6 break. Queen B6. Rook C2. These next few piece moves by white. I really like. Rook C1 opens up this square for the queen. All right. Queen D4. Black would like to get an exchange in. Queen exchange may make it uh a little bit easier possibly to get in a B5 break. Uh possible doubling up and then B5. White does not want a queen exchange. Plays H3 here. So not so quick to get out of this uh get out of this pin. Takes some time out to prevent any coordination. let's say on f2. So this is the moment to flick in the flight square. Follow up rook c7. Queen c1. And now black better watch because this queen is very short on squares. White's ready to take over full control of this diagonal. So queen aborts here. Flies away before she's going to get uh trapped. Bishop E3. Queen B8. Notice the queen staying in a position to still maybe be there in support. B5 break. Follow up. Bishop D4. Queen D1. Bishop exchange. And look at how quickly things have turned. It was just a moment ago where it was the black queen who was in the center. Now it is the white queen. And what a piece this is. Fantastic post. Not so easily challenged. Now, at this point, black goes with knight H to F6. Gets back into the ball game. Let's revisit B5 in this position. There is a beautiful tactical sequence. If black tries B5 here, I almost didn't uh test out B5 in this position. I'm glad I did. Check out this tactical sequence. If black goes with B5, white can take on B5. A takes B and not knight captures B, that would hang the rook. So you take with the bishop. And now what you always have to be uh careful about in these positions are the uh rook sacrifice on the knight. this rook for two minor pieces combination. It's not working out here. Let's see why. If black goes for rook captures knight, there's rook captures rook. And after queen captures bishop, you ready for this one? There would be a beautiful shot with rook captures E7. And we're not done yet. How would this look? After rook captures rook, we would see a back rank check. After a knight block, we would see checkmate. Rook captures knight and queen corner pocket. Game over. That's a very pretty uh sequence right there. So B5 going all the way back here. B5 yet again not working with this queen having a clear view of that H8 square. So knight H to F6 maybe now it's nearby. All right from here white says what are you doing about E6 or E7? Queen pulled away defending very cluttered. White with the space advantage. That space advantage uh well that space I should say is certainly felt. Space is an advantage in this position. Followup. Rook E3. Rook C8. Small improvements. King H2. King F8. G4. Okay, maybe we're ready to scare the knight away. Queen B6. And at this point, uh, white is okay going for a queen exchange. The computer does not see this as best, but uh, yeah, wants to go with queen f4, queen d2, keep the queens on board, but maybe a practical decision with this time format. Welcomes the endame where white still has a slight uh, pull. So we get a queen exchange. King improvement followup H4. H6. Bishop E2. White wants G5. Doesn't want to deal with knight H5. All right. King G7 is played at this point. Uh at the top the computer says play G5. Uh try to hold up the G5 idea. Uh how would this look if G5 is played? You can take on G5. You could play F4. In the end, there is a square for black to work with on E5. So either knight could maybe occupy this point. This square will not be available shortly for the knight. White will be able to establish a pawn, another invasive pawn in the position. But how black plays it does not insert G5. King G7 it is. Now we see G5. The exchange on G5 and knight H7 is played. Now if you go with knight H5, this isn't any good because you end up with an isolated pawn that can be immediately targeted and it's not so easy to hang on to that. So keep in mind these rooks tethered to the defense of E7. They are not having fun. Neither is this B pawn. You don't have to worry about B5 with the knight on B6. The other detail, this seems like a convenient way to defend the pawn. Uh this weakened pawn, isolated pawn, but you'd be looking at this maneuver into F4 for the knight. So, uh, not a good idea to take on the isolated pawn. We have knight H7 in this game. Followup F4. And notice again looking back at that uh, opportunity for black to play G5 rather than king G7. In that variation, black would have had access to E5. Now, you don't. Black has or white here has control over both of these black squares. F6 and E5. The knights are running out of squares. Very good coordination space everywhere for white in the center on the king's side. A nice uh square has opened up on G4 for the bishop. Full control for white. Now at this stage we see knight C8. computer saying uh at the top it's looking at f6, but even with this immediate challenge of g5 uh it prefers white. We're looking at something uh about plus 1.5. So still very good for white even with this top move f6. In this one, it is knight C8, bishop G4, the B5 break is finally there, but by this point, too late. Move 36, B5. Something uh has clearly gone wrong. If it's taking that long to get in this break, how it follows, C takes B, A takes B, knight takes B. Black's relying on a fork in the end, but this knight also has a fork in mind. It is supported on B5. Rook captures D. There would follow knight C7. And there you go. Both rooks targeted in the game. It's knight B6. There is a pin in the position or has been a pin for some time. And now that D6 is not defended with the knight, knight D6 is available. And where do you go with the rook in the game? Rook F8. If you go on the counterattack uh yet again there would be another fork. So rook f8 it is position clearly collapsing for black. Knight takes dook on seven to e5 pin. Rook check. Bishop blocks. Knight is hit with the rook. Still needs to find a new home. Rook E3. Everything under control. Base point covered. No longer a pin of the bishop. Rook C1. Back to the seventh rank. Rook C5. Rook B7. And after knight D5, we have rook E5. And black has finally had enough resigns. Completely tied up in a knot. This final position. Not a single black piece in white's camp. This knight is going to fall shortly. Look at how invasive uh the white pieces pawns are in this position. Two extra pass pawns, connected passers. No hope for black. So very nice controlled game out of this Kings Indian defense with that uh light square bishop for king knight. uh imbalance. Anyhow, your thoughts on this one. Feel free, as usual, to leave any feedback to this video in the comment section below. Hope you enjoyed it. It maybe took a thing or two away. That's all for now. Take care. Bye. 
            """"
        ]
        
        self.hikaru_examples = [
            """
                Welcome back everyone. For today's video, we are going to be taking a look at the early title Tuesday on chess.com being played today, July 22nd of the great year 2025. Now, yesterday the freestyle event ended in Las Vegas. I drove back to California and as always, we're back to playing the premier online event. Now, over my illustrious career of playing in Title Tuesday, I have won a whopping 104 of these events. And today I'm trying to make it 105. Now rather shockingly, one of the rare occurrences that we see is that the current world chess champion, Gesh Maraju, is actually playing in this event. He has played title Tuesday before, but it's very rare to see him playing. But Gh is the world champion. He's performed very well, and it's no surprise that I'm paired up with him in the 10th round of this tournament. Now, going into this critical 10th round matchup, both of us have eight points out of nine. We both have suffered an early loss in the tournament, but we're tied for the lead. And the winner of this game most likely is going to go on and win the event. So Gesh has the white pieces. He starts out with knight f3. We get d5 g3 g6 bishop g2 bishop g7. And now we get the move pawn to d4. Now guesh could have castled the king here but he doesn't want to fall prey to the very famous meme also known as the big black center. I get these pawns in the center of the board. White is still fine after a move like d3. But nonetheless the big black center is really really powerful. So, Gesh goes D4 to stop me from playing E5. I go knight D7. Castles knight F6. C4 takes knight A3. And now I move my knight to try and temporarily guard the juicer on C4. Gh sacks the horse. And now he plays queen to A4, checking the king and attacking the pony. I go C6 takes. And now we get to move bishop to E6. Gh plays queen C2. And here I play bishop F5, tickling the queen. We get queen B3, queen B6. And now Gash plays move E3. Now, it's rather surprising that at this point Gesh spends 30 seconds on this move individually. Gh is one of the best prepared players in the world. And what this tells me is that Gesh was a little bit unfamiliar with this position. So, we get E3. I castle bishop D2. And now I play the move bishop to E4. Now, what I'm doing here is creating the very classic wooden shield in honor of XQC with this bishop in the middle of the board. But more importantly, what white would love to do thematically is move this pony and try to open up the long diagonal for this op on G2. So I go bishop to E4. And now if white tries to move the bishop, you hang the knight. And if you move the knight, I can always just trade the bishops. And now you no longer have this Fon Cheetoed bishop spying the diagonal. So here we got the move queen to A3. I play the move queen to C7, retreating the queen to guard the pawn on E7. Gash goes rook C1. I play rook to D8. And as soon as I played this move, I realized that this was a very slightly inaccurate move. Now, the reason I played rook D8 here is I thought with the rook on D8, I can potentially push E5 here, opening up the D file or a move like C5, and I figure there's going to be a ton of pressure on the open D line. But after I go rook to D8, Gesh plays this move knight to E5. I trade. And what I realized almost immediately is that there's a very nasty trap here. If I play a random move like let's just say pawn to E6. White can go bishop to A5 here attacking the queen and the rook on the diagonal. Queen guards the bishop. And if I play B6, white can go knight takes pawn. The rook guards the pony. And if I take the bishop after knight to E7, this is the classic fossil here. The knight checks the king. The queen guards the knight and the rook targets the queen at the same time. I'd have to do a bow test gambit giving up the queen. But here white has a queen for a knight and a bishop. And last time that I checked, even though I learned before common core math was a thing, nine is worth more than six. So I trade the bishops. And now I play this move, bishop to F8 after a very long think. Now this move is a typical blitz move. In a classical game, I almost certainly would not play it. But in a blitz game, I'm spending a lot of time. I'm not sure what to do. And I when I play this move, I'm essentially calling Guash's blood. Now, you'll notice here that I'm down 32 seconds on the clock. For for players who are beginners or am amateur players, don't worry about time. Time is not the end of the world. But at the top level, time plays a very critical role when you get towards the end of the game. And usually if you have a small time advantage on the clock, it is a massive advantage. So when I play bishop to F8 here, I'm basically asking Gh what is he going to do? Now here, Gash spends, I believe, one minute in this position before playing the move B4. And this really highlights one of the differences between players like Guekesh who are more about pure calculation and more intuitive players like myself or Pragnanta or people like Hans Nean for example because when I play bishop to F8, Ghesh here tries to calculate the entire line after bishop to A5 B6 and now there are all these different possibilities. Now if white goes rook takes C6 after queen B7 the bishop is hanging. There's also a threat on this diagonal as well because the rook is pinned. And if white moves the bishop to let's say B4 after a5 let's say bishop to E1 here I can go knight to D7 and suddenly this tower is under severe pressure due to the pin. White in fact is just losing here. If white plays move like queen C3 I can simply I can simply play the move takes um and after takes takes and the move rook a c8 the rook on c6 gets pinned and I win the game. So white has to use a lot of time here trying to figure out what you're doing. Now the computer of course says that in this position if white plays the other order with queen to b3 attacking the pawn on f7 here if I play the move pawn takes bishop now white can take the pawn after king h8 rook takes c6 queen b7 and king g1 computer says that white is better here even though white is down on bishop for a couple of central pawns because you can go for the double stack the queen and the knight are very close to the king I'm not actually sure I believe that white is objectively better but the computer thinks there are chances now can you play this in a blitz game where you can't calculate everything to the end. You kind of have to make the decision. Do you trust your intuition that there's something? And if you don't see it, then you play something else. What you going to do? Now, Gh spends a minute and ultimately decides to play the move B4 instead. And there's nothing wrong with this move specifically. But the problem is that because Gash spends 1 minute on the clock here. He's now on his back foot. He's down 30 seconds here. And my moves, as you're about to see, become a lot easier. So, I play knight D7. Gosh plays F4. I go E6. And now we get the move queen B3. Here I play the move knight back to F6. And the main reason I play this move is that Gesh has gone for the classic stonewall setup with the pawns on D4, E3, and F4. Now a stonewall can sometimes be very very strong, but in this position, I'm trusting that with the Stonewall and these weak squares for the knight to jump, specifically E4, and D5, I'd rather keep the knights on the board. So here we get the move queen D3. And now I go queen D6. And all my moves here, I'm playing with float. It's very easy. I want to play on the light squares here. 
            """,
            """
                I'm not flying to Saudi Arabia anytime soon. No, I'm going to Las Vegas for the uh freestyle event. Actually, that's one of the things. I was invited to go to Saudi Arabia, but why would I fly halfway around the world to Saudi Arabia um and then have to fly to Las Vegas, which is I mean, okay, I don't want to say serious, but I mean, it's a real tournament. Someone said play the Grand Prix. So, I'm going to play Sicilian. Why not? Let's go here. I'm going to go B5 B4 immediately. Go here. B5 B4. Of course, go B5. B4. I mean, A5 A4 feels a little bit slow, but I'm still going to do it anyway. Oh, he does go D4. I didn't think he would play D4. Okay, so I'll take go here. Trade the knights. I really didn't think he was going to play D4. I'm kind of surprised by that move. It's not that bad, but I'm surprised by it. Now I have to come with ideas though. Um, where I'll put my queen. I do have A3 somewhere. Um, I don't actually like my position. Huh. Okay, let's think about this for a second. I'll just go here to guard the knight. I don't like my position here. I'm not going to lie. I'm not happy with the position. It goes knight f2. I am happy to see that though cuz now I can play knight to f6 and on e5 I just take and I'm fine. Maybe I should have put the queen on c8 though cuz c5 is still kind of pesky potentially. Here goes rook d1. I was a little bit worried about this move. Um H go here. I don't like my position at all actually. I just take. Not a position I'm happy with, but if I can hang on for a few more moves, I should be fine. Let's see what he does. Okay, now I get castled. Now I'm not worried at all. Now, now we just play chess and I should be very happy. G4. I kind of expected that. Um, knight D7. Oh, great. Did I just blunder E5 maybe? should have got knight a5, I think. Okay, trades. I'm very happy to see that, of course, cuz E5 surely isn't all that good. He does have knight D3 maybe. Yeah, he goes knight D3 now. I could play B3 here. Queen B7 actually looks very thematic, but queen B7 E5. No, that's not a move cuz E5 I take take and I'm fine. It does play. Wait, so takes. I mean, I'm just guarding everything here. Unless I'm insane. Maybe I should. It's going to have knight C. No, knight C5. Rook C7 should be okay. Okay, we should get a win. Let's go. E4. Attack the knight. Open up the scope. Take the pawn. D1. Okay, I can play rook C8 here. Attack the bishop. attack the bishop. That's actually a good move. Um, wait a take. Rook A2. Should be a free pawn. Just making sure there are no checkmates here. No, I go here. And now this is how my opponent feels. That's how my opponent feels. He resigns. Okay, we got the win one out of one. Thank Quaver 29 months. Fuzzy for the prime. Thank you so much. All right, we got the win one out of one. Let's watch Ali Resa Fuja. He is playing against Ricky. He is ahead of material. He's going to win. Okay, let's take a look at other games. Hans is going to win. Okay, let's move on. Uh Jeffrey is better but not cleanly winning here. G takes H6. This is a draw with correct play. This is a draw with correct play. Yeah, Jeffrey's going to draw in round one. Wow. Knight B4. Wow. like B3. I mean, this should be a draw. I mean, all white has to do here is get the um knight A6 maybe is a draw, but I think you just go knight D3 and you just bring the king to the center like Knight F4 and bring your king B4 also good enough as well. B4. Yeah. Oh, wait. No, there's King E6. Wait. Oh, we have knight C5. Knight B3. Okay. Knight F4. Knight F4. This was actually a draw, though. He He missed um Knight C5 and Knight B3 was just winning the pawn. Jeffrey might actually get the win now. Yeah, Jeffrey's going to probably win now. Title Tuesday starts at 4:00. It used to be 5:00, but it's 4:00. Yeah, Jeffrey's going to win now. Um Hans got the win. Okay, as expected. What else do we have? Fabiano is probably should sack the rook. King C4. Knight B5. E7. Knight D6. Knight D6. Yeah, now it's just a box. Knight E4. I mean, you should just go knight E4. Isn't this just a box? I mean, this is the L that you learn. And I mean, yeah, H3 is just winning here. You go H2 and then you go. Okay. Confused. Confused by Fabiano's game. Thanks so much to Day Bass Bass or Day Badass for the Prime. Thank you so much. Appreciate it. Little bit confused by that, but it's round one. Little bit confused. Okay. What's the scariest city you've been to globally? Um, I'm not sure I want to answer that question. Uh, I'm not sure I should answer that question cuz somebody's People are going to be upset if I say that. Um, but I I mean I don't know. I I think people are going to be be unhappy. But it's a but but I'll say this. It's a city. It's a city uh it's a city that I've been to a lot. Let's let's leave it at that. Okay. Let's leave it at that. It's a city I've been to a lot. So, let's see. We go. So we get king d6. Um what is king d king king d8? Just checkmate. Um. Huh. Strange. Okay. Very strange. I'm going to Japan in April. It's definitely not Japan. I'll tell you guys that. It's definitely not Japan. Wow. Eric is solo ray that I get him around to. I I I've been playing B3 a lot. I'm going to play D4. MKS for the prime. I mean D4. I mean, all the openings are fine. Depends what kind of game you want. Um I should have just played knight F3 G3. Actually, I'm playing Eric. Okay, so he's trying to play something standard. I'm going to play queen C2. It's a little bit unusual, but it's a line. Let's go here. Big bad Krammnik actually was playing the setup setup back in the heyday. Okay, C6 is definitely not correct. I mean I have E4 E5 takes. I also have eight. Oh, so he's trying to play Okay, so so he's trying to play Cambridge Springs, I guess. Okay, this is very weird. very very weird position. [Music] I mean, he should take the pawn, but in general, I'm really happy with these structures. 
            """
        ]
        
        self.combined_prompts = {
            "opening": """
You are a chess streamer on youtube combining the educational style of Chess Network with the entertaining personality of GM Hikaru Nakamura. 

Analyze this game and provide commentary that:
1. Explains the chess principles involved (Chess Network style)
2. Adds entertaining observations and quick assessments (Hikaru style)
3. Mentions both players' plans and ideas

Game: {white} vs {black}
Moves: {moves}
Current position: {position}

Make the commentary engaging while being educational.
""",
            
            "middlegame": """
You are a chess commentator combining Chess Network's analytical approach with Hikaru's entertaining style.

This is a middlegame position with tactical opportunities. Provide commentary that:
1. Identifies the key pieces and their roles
2. Explains the strategic themes
3. Uses Hikaru's quick pattern recognition style
4. Points out critical moments with excitement

Game: {white} vs {black}
Recent moves: {moves}
Position: {position}
Critical move: {critical_move}

Balance education with entertainment.
""",
            
            "endgame": """
You are commenting on an endgame as a combination of Chess Network's precision and Hikaru's assessment skills.

Provide commentary that:
1. Explains endgame principles clearly
2. Assesses who is winning and why
3. Uses Hikaru's direct evaluation style
4. Mentions technique and precision needed

Game: {white} vs {black}
Endgame moves: {moves}
Position: {position}
Game outcome: {result}

Make it both educational and engaging.
"""
        }
    
    def generate_move_commentary(self, move: ChessMove, metadata: Dict, game_phase: str = "middlegame") -> str:
        """Generate commentary for a specific move"""
        
        if game_phase not in self.combined_prompts:
            game_phase = "middlegame"
        
        # Format the moves for the prompt
        moves_text = ""
        if move.white_move and move.black_move:
            moves_text = f"{move.move_number}. {move.white_move} {move.black_move}"
        elif move.white_move:
            moves_text = f"{move.move_number}. {move.white_move}"
        elif move.black_move:
            moves_text = f"{move.move_number}... {move.black_move}"
        
        # Create the prompt
        prompt = self.combined_prompts[game_phase].format(
            white=metadata['white'],
            black=metadata['black'],
            moves=moves_text,
            position=move.position_after,
            critical_move=move.white_move or move.black_move,
            result=metadata.get('result', '*')
        )
        
        return prompt
    
    def analyze_game_phases(self, moves: List[ChessMove]) -> Dict[str, List[int]]:
        """Determine game phases for better commentary"""
        total_moves = len(moves)
        
        # Simple heuristic for game phases
        opening_end = min(10, total_moves // 3)
        middlegame_end = min(25, 2 * total_moves // 3)
        
        phases = {
            'opening': list(range(0, opening_end)),
            'middlegame': list(range(opening_end, middlegame_end)),
            'endgame': list(range(middlegame_end, total_moves))
        }
        
        return phases
    
    def get_commentary_intensity(self, move: ChessMove) -> str:
        """Determine commentary intensity based on move type"""
        if move.move_type == "checkmate":
            return "high"
        elif move.move_type in ["check", "capture"] or move.is_critical:
            return "medium"
        else:
            return "low"

# Initialize the commentary generator
print("Chess Commentary Generator initialized!")

## Commentary Style Examples

Here are example commentaries that capture the essence of Chess Network and GM Hikaru Nakamura:

### Chess Network Style:
- Educational and analytical
- Clear explanations of chess principles
- Focuses on why moves are good or bad
- Patient and methodical approach

### GM Hikaru Style:
- Entertaining and spontaneous
- Quick pattern recognition
- Chat interaction references
- Mix of serious analysis with casual commentary
- "That's just winning" type expressions

In [ ]:
@dataclass
class ChessMove:
    """Represents a chess move with analysis"""
    move_number: int
    white_move: Optional[str]
    black_move: Optional[str]
    position_before: str  # FEN
    position_after: str   # FEN
    evaluation: Optional[float] = None
    is_critical: bool = False
    move_type: str = "normal"  # "normal", "blunder", "brilliant", "sacrifice", etc.

class ChessGameParser:
    """Parses PGN format chess games"""
    
    def __init__(self):
        pass
    
    def parse_pgn(self, pgn_string: str) -> Tuple[Dict, List[ChessMove]]:
        """
        Parse a PGN string and return game metadata and moves
        """
        pgn_io = io.StringIO(pgn_string)
        game = chess.pgn.read_game(pgn_io)
        
        if not game:
            raise ValueError("Invalid PGN format")
        
        # Extract metadata
        metadata = {
            'white': game.headers.get('White', 'Unknown'),
            'black': game.headers.get('Black', 'Unknown'),
            'result': game.headers.get('Result', '*'),
            'date': game.headers.get('Date', 'Unknown'),
            'event': game.headers.get('Event', 'Unknown'),
            'opening': game.headers.get('Opening', 'Unknown')
        }
        
        # Parse moves
        moves = []
        board = game.board()
        move_number = 1
        
        for i, move in enumerate(game.mainline_moves()):
            position_before = board.fen()
            
            # Determine if this is a white or black move
            is_white_move = board.turn == chess.WHITE
            
            # Make the move
            board.push(move)
            position_after = board.fen()
            
            # Create move object
            if is_white_move:
                # Starting a new move pair
                chess_move = ChessMove(
                    move_number=move_number,
                    white_move=board.san(move),
                    black_move=None,
                    position_before=position_before,
                    position_after=position_after
                )
                moves.append(chess_move)
            else:
                # Completing the move pair
                if moves and moves[-1].move_number == move_number:
                    moves[-1].black_move = board.san(move)
                    moves[-1].position_after = position_after
                else:
                    # Edge case: game starts with black move
                    chess_move = ChessMove(
                        move_number=move_number,
                        white_move=None,
                        black_move=board.san(move),
                        position_before=position_before,
                        position_after=position_after
                    )
                    moves.append(chess_move)
                move_number += 1
        
        return metadata, moves
    
    def analyze_move_criticality(self, moves: List[ChessMove]) -> List[ChessMove]:
        """
        Analyze moves to identify critical moments (simplified heuristic)
        """
        for i, move in enumerate(moves):
            # Simple heuristics for identifying critical moves
            # In a real implementation, you'd use a chess engine
            
            # Check for captures, checks, or piece sacrifices
            if move.white_move:
                if 'x' in move.white_move or '+' in move.white_move or '#' in move.white_move:
                    move.is_critical = True
                    if 'x' in move.white_move and any(piece in move.white_move for piece in ['Q', 'R', 'B', 'N']):
                        move.move_type = "capture"
                    elif '+' in move.white_move:
                        move.move_type = "check"
                    elif '#' in move.white_move:
                        move.move_type = "checkmate"
            
            if move.black_move:
                if 'x' in move.black_move or '+' in move.black_move or '#' in move.black_move:
                    move.is_critical = True
                    if 'x' in move.black_move and any(piece in move.black_move for piece in ['Q', 'R', 'B', 'N']):
                        move.move_type = "capture"
                    elif '+' in move.black_move:
                        move.move_type = "check"
                    elif '#' in move.black_move:
                        move.move_type = "checkmate"
        
        return moves

# Test the parser
parser = ChessGameParser()
print("Chess game parser initialized successfully!")

In [ ]:
import re
import json
from typing import List, Dict, Tuple, Optional
import openai
from dataclasses import dataclass
import chess
import chess.pgn
import io

# Install required packages if not already installed
try:
    import chess
except ImportError:
    !pip install python-chess

try:
    import openai
except ImportError:
    !pip install openai

# Chess Commentary Generator

This notebook creates commentary for chess games in the combined styles of Chess Network and GM Hikaru Nakamura.

## Features:
- Parse chess games in PGN format
- Analyze positions and moves
- Generate commentary combining Chess Network's educational approach with Hikaru's entertaining personality
- Support for various commentary styles and intensities